<a href="https://colab.research.google.com/github/SIVASKANDAN-P/Automatic-Ticket-Classification/blob/main/Automatic_Ticket_Classification_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Problem Statement

You need to build a model that is able to classify customer complaints based on the products/services. By doing so, you can segregate these tickets into their relevant categories and, therefore, help in the quick resolution of the issue.

You will be doing topic modelling on the <b>.json</b> data provided by the company. Since this data is not labelled, you need to apply NMF to analyse patterns and classify tickets into the following five clusters based on their products/services:

* Credit card / Prepaid card

* Bank account services

* Theft/Dispute reporting

* Mortgages/loans

* Others


With the help of topic modelling, you will be able to map each ticket onto its respective department/category. You can then use this data to train any supervised model such as logistic regression, decision tree or random forest. Using this trained model, you can classify any new customer complaint support ticket into its relevant department.

## Pipelines that needs to be performed:

You need to perform the following eight major tasks to complete the assignment:

1.  Data loading

2. Text preprocessing

3. Exploratory data analysis (EDA)

4. Feature extraction

5. Topic modelling

6. Model building using supervised learning

7. Model training and evaluation

8. Model inference

## Importing the necessary libraries

In [1]:
import json
import numpy as np
import pandas as pd
import re, nltk, spacy, string
import en_core_web_sm
nlp = en_core_web_sm.load()
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from plotly.offline import plot
import plotly.graph_objects as go
import plotly.express as px

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from pprint import pprint

## Loading the data

The data is in JSON format and we need to convert it to a dataframe.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Opening JSON file
f = open(r'/content/drive/MyDrive/ticket classification/complaints-2021-05-14_08_16.json') # Write the path to your data file and load it

# returns JSON object as
# a dictionary
data = json.load(f)
complaint=pd.json_normalize(data)

## Data preparation

In [4]:
# Inspect the dataframe to understand the given data.
complaint.head()

,_index,_type,_id,_score,_source.tags,_source.zip_code,_source.complaint_id,_source.issue,_source.date_received,_source.state,...,_source.company_response,_source.company,_source.submitted_via,_source.date_sent_to_company,_source.company_public_response,_source.sub_product,_source.timely,_source.complaint_what_happened,_source.sub_issue,_source.consumer_consent_provided
0,complaint-public-v2,complaint,3211475,0.0,None,90301,3211475,Attempts to collect debt not owed,2019-04-13T12:00:00-05:00,CA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-13T12:00:00-05:00,None,Credit card debt,Yes,,Debt is not yours,Consent not provided
1,complaint-public-v2,complaint,3229299,0.0,Servicemember,319XX,3229299,Written notification about debt,2019-05-01T12:00:00-05:00,GA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-05-01T12:00:00-05:00,None,Credit card debt,Yes,Good morning my name is XXXX XXXX and I apprec...,Didn't receive enough information to verify debt,Consent provided
2,complaint-public-v2,complaint,3199379,0.0,None,77069,3199379,"Other features, terms, or problems",2019-04-02T12:00:00-05:00,TX,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-02T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,I upgraded my XXXX XXXX card in XX/XX/2018 and...,Problem with rewards from credit card,Consent provided
3,complaint-public-v2,complaint,2673060,0.0,None,48066,2673060,Trouble during payment process,2017-09-13T12:00:00-05:00,MI,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2017-09-14T12:00:00-05:00,None,Conventional home mortgage,Yes,,None,Consent not provided
4,complaint-public-v2,complaint,3203545,0.0,None,10473,3203545,Fees or interest,2019-04-05T12:00:00-05:00,NY,...,Closed with explanation,JPMORGAN CHASE & CO.,Referral,2019-04-05T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,,Charged too much interest,N/A


In [5]:
#print the column names
complaint.columns

Index(['_index', '_type', '_id', '_score', '_source.tags', '_source.zip_code',
       '_source.complaint_id', '_source.issue', '_source.date_received',
       '_source.state', '_source.consumer_disputed', '_source.product',
       '_source.company_response', '_source.company', '_source.submitted_via',
       '_source.date_sent_to_company', '_source.company_public_response',
       '_source.sub_product', '_source.timely',
       '_source.complaint_what_happened', '_source.sub_issue',
       '_source.consumer_consent_provided'],
      dtype='object')

In [6]:

#Assign new column names
rename_cols = [i[1:] for i in complaint.columns] # removed the first underscore
rename_col_fin=[]
for i in rename_cols:
  if i.find('source') != 1:
    i = i.replace('source.','')
  rename_col_fin.append(i)
rename_dict = dict(zip(complaint.columns,rename_col_fin))
complaint = complaint.rename(columns = rename_dict)

In [7]:
complaint.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78313 entries, 0 to 78312
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   index                      78313 non-null  object 
 1   type                       78313 non-null  object 
 2   id                         78313 non-null  object 
 3   score                      78313 non-null  float64
 4   tags                       10900 non-null  object 
 5   zip_code                   71556 non-null  object 
 6   complaint_id               78313 non-null  object 
 7   issue                      78313 non-null  object 
 8   date_received              78313 non-null  object 
 9   state                      76322 non-null  object 
 10  consumer_disputed          78313 non-null  object 
 11  product                    78313 non-null  object 
 12  company_response           78313 non-null  object 
 13  company                    78313 non-null  obj

In [8]:
complaint.shape

(78313, 22)

In [9]:
#Assign nan in place of blanks in the complaints column
complaint.isnull().sum()
complaint.complaint_what_happened = complaint.complaint_what_happened.apply(lambda i: np.nan if len(i.replace(" ",'')) == 0 else i)
complaint.complaint_what_happened.isnull().sum()

57241

In [10]:
#Remove all rows where complaints column is nan
complaint = complaint.dropna(axis=0,subset='complaint_what_happened')
complaint.shape

(21072, 22)

In [11]:
#complaint[[!(complaint.complaint_what_happened.isnull())]]
#complaint = complaint.dropna(axis=0,subset='complaint_what_happened')

In [12]:
# droping columns with more that 0.8 % null values
drop_cols = list(complaint.isnull().sum()[complaint.isnull().sum() > 0.8*78313].index)
complaint = complaint.drop(columns = list(complaint.isnull().sum()[complaint.isnull().sum() > 0.8*78313].index))

In [13]:
complaint.isnull().sum()

index                            0
type                             0
id                               0
score                            0
tags                         17256
zip_code                      4645
complaint_id                     0
issue                            0
date_received                    0
state                          143
consumer_disputed                0
product                          0
company_response                 0
company                          0
submitted_via                    0
date_sent_to_company             0
company_public_response      21070
sub_product                   2109
timely                           0
complaint_what_happened          0
sub_issue                     8176
consumer_consent_provided        0
dtype: int64

In [14]:
#complaint.type.value_counts()

In [15]:
#complaint.index.value_counts()

In [16]:
#complaint.id.value_counts()

In [17]:
#complaint.score.value_counts()

In [18]:
#drop_cols.append(['id','score','index','type'])
#complaint = complaint.drop(columns = ['id','score','index','type'])

In [19]:
#complaint.head()

In [20]:
#complaint.company_public_response.apply(lambda i: np.nan if len(str(i).replace(" ",'')) == 0 else str(i)).value_counts()

In [21]:
#drop_cols.append('company_public_response')
#complaint = complaint.drop(columns = 'company_public_response')

In [22]:
#complaint.isnull().sum()

In [23]:
#complaint.tags = complaint.tags.fillna(value='None')

In [24]:

#complaint = complaint.dropna(axis = 0,subset=['state'])


In [25]:
#complaint.isnull().sum()

In [26]:
# zip code doesnt seem to help in identifying the ticket type
#drop_cols.append('zip_code')
#complaint = complaint.drop(columns = ['zip_code'])


In [27]:
#complaint.sub_issue.value_counts()

In [28]:

#complaint = complaint.dropna(axis = 0,subset=['sub_product'])
#complaint.shape

In [29]:
#complaint = complaint.dropna(axis = 0,subset=['sub_issue'])
#complaint.shape

In [30]:
#complaint.isnull().sum()

In [31]:
#drop_cols

In [32]:
complaint.head()

,index,type,id,score,tags,zip_code,complaint_id,issue,date_received,state,...,company_response,company,submitted_via,date_sent_to_company,company_public_response,sub_product,timely,complaint_what_happened,sub_issue,consumer_consent_provided
1,complaint-public-v2,complaint,3229299,0.0,Servicemember,319XX,3229299,Written notification about debt,2019-05-01T12:00:00-05:00,GA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-05-01T12:00:00-05:00,None,Credit card debt,Yes,Good morning my name is XXXX XXXX and I apprec...,Didn't receive enough information to verify debt,Consent provided
2,complaint-public-v2,complaint,3199379,0.0,None,77069,3199379,"Other features, terms, or problems",2019-04-02T12:00:00-05:00,TX,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-02T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,I upgraded my XXXX XXXX card in XX/XX/2018 and...,Problem with rewards from credit card,Consent provided
10,complaint-public-v2,complaint,3233499,0.0,None,104XX,3233499,Incorrect information on your report,2019-05-06T12:00:00-05:00,NY,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-05-06T12:00:00-05:00,None,Other personal consumer report,Yes,Chase Card was reported on XX/XX/2019. However...,Information belongs to someone else,Consent provided
11,complaint-public-v2,complaint,3180294,0.0,None,750XX,3180294,Incorrect information on your report,2019-03-14T12:00:00-05:00,TX,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-03-15T12:00:00-05:00,None,Credit reporting,Yes,"On XX/XX/2018, while trying to book a XXXX XX...",Information belongs to someone else,Consent provided
14,complaint-public-v2,complaint,3224980,0.0,None,920XX,3224980,Managing an account,2019-04-27T12:00:00-05:00,CA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-27T12:00:00-05:00,None,Checking account,Yes,my grand son give me check for {$1600.00} i de...,Funds not handled or disbursed as instructed,Consent provided


In [33]:
complaint.shape

(21072, 22)

## Prepare the text for topic modeling

Once you have removed all the blank complaints, you need to:

* Make the text lowercase
* Remove text in square brackets
* Remove punctuation
* Remove words containing numbers


Once you have done these cleaning operations you need to perform the following:
* Lemmatize the texts
* Extract the POS tags of the lemmatized text and remove all the words which have tags other than NN[tag == "NN"].


In [ ]:
#def remsquarbr(com):
com = "st[sdsd] heldl[jhkj] sdafs.? [hello]"
st=""
for x in re.findall("\[[\w]*\]",com):
  com = com.replace(x,'')
# removing special character
for x in re.findall("[^a-zA-Z0-9_ ]",com):
  com = com.replace(x,'')
print(com)
#re.search("\]",'st[sds][]')
  #x.span()[0]
for x in com.split():
    print(re.search("[0-9]",x))

st heldl sdafs 
None
None
None


In [34]:
com = "st[sdsd] heldl[jhkj] sdafs123.? [hello]"
st=''
for x in com.split():
    if re.search("[0-9]",x)==None:
      st = st+' '+x
    else:
      continue
print(st)

 st[sdsd] heldl[jhkj] [hello]


In [52]:
def textprocess(com):
  # convert text to lower case
  com = com.lower()
  #Remove text in square brackets
  for x in re.findall("\[[\w]*\]",com):
    com = com.replace(x,'')
  # removing special character
  for x in re.findall("[^a-zA-Z0-9_ ]",com):
    com = com.replace(x,'')
  # removing words with numbers
  st=''
  for x in com.split():
    if re.search("[0-9]",x)==None:
      st = st+' '+x
    else:
      continue
  return st.strip()



In [ ]:
textprocess('hellow [12023] dssdf122 122sdsdf  sdfasd[sadf]sd')

' hellow sdfasdsd'

In [36]:
# Write your function here to clean the text and remove all the unnecessary elements.
#complaint.complaint_what_happened = complaint.complaint_what_happened.apply(lambda x: x.lower())
complaint.complaint_what_happened = complaint.complaint_what_happened.apply(lambda x: textprocess(x))

In [41]:
print(complaint.complaint_what_happened.head())

1      good morning my name is xxxx xxxx and i appre...
2      i upgraded my xxxx xxxx card in and was told ...
10     chase card was reported on however fraudulent...
11     on while trying to book a xxxx xxxx ticket i ...
14     my grand son give me check for i deposit it i...
Name: complaint_what_happened, dtype: object


In [58]:
#Write your function to Lemmatize the texts
def lemmaextracter(sent):
    nlp1 = en_core_web_sm.load(disable=['ner'])
    doc = nlp1(sent)
    lis = ""
    for tok in doc:
        #if (tok.pos_ == 'NOUN' or tok.pos_ == 'PROPN') and (len(tok) != 1):
        lis = lis + ' ' + tok.lemma_
    return lis.strip()

In [60]:
#lemmaextracter("hello how are you doing?")
complaint['complaint_what_happened_lemma'] = complaint.complaint_what_happened.apply(lambda x : lemmaextracter(x))

KeyboardInterrupt: ignored

In [ ]:
complaint.complaint_what_happened_lemma.head()

In [ ]:
#Create a dataframe('df_clean') that will have only the complaints and the lemmatized complaints
df_clean = complaint[['complaint_what_happened','complaint_what_happened_lemma']]
df_clean = df_clean.rename({'complaint_what_happened' : 'complaint', 'complaint_what_happened_lemma' : 'complaint_lemma'})

In [ ]:
df_clean.head()

In [ ]:
#Write your function to extract the POS tags

def pos_tag(text):
  # write your code here
  #nlp1 = en_core_web_sm.load(disable=['ner'])
  doc = nlp(text)
  lis = ""
  for tok in doc:
    if (tok.pos_ == 'NOUN' or tok.pos_ == 'PROPN') and (len(tok) != 1):
      lis = lis + ' ' + tok.lemma_
  return lis.strip()




df_clean["complaint_POS_removed"] = df_clean.complaint_lemma.apply(lambda x: pos_tag(x)) #this column should contain lemmatized text with all the words removed which have tags other than NN[tag == "NN"].


In [ ]:
#The clean dataframe should now contain the raw complaint, lemmatized complaint and the complaint after removing POS tags.
df_clean

## Exploratory data analysis to get familiar with the data.

Write the code in this task to perform the following:

*   Visualise the data according to the 'Complaint' character length
*   Using a word cloud find the top 40 words by frequency among all the articles after processing the text
*   Find the top unigrams,bigrams and trigrams by frequency among all the complaints after processing the text. ‘




In [ ]:
# Write your code here to visualise the data according to the 'Complaint' character length

#### Find the top 40 words by frequency among all the articles after processing the text.

In [ ]:
#Using a word cloud find the top 40 words by frequency among all the articles after processing the text


In [ ]:
#Removing -PRON- from the text corpus
df_clean['Complaint_clean'] = df_clean['complaint_POS_removed'].str.replace('-PRON-', '')

#### Find the top unigrams,bigrams and trigrams by frequency among all the complaints after processing the text.

In [ ]:
#Write your code here to find the top 30 unigram frequency among the complaints in the cleaned datafram(df_clean).


In [ ]:
#Print the top 10 words in the unigram frequency


In [ ]:
#Write your code here to find the top 30 bigram frequency among the complaints in the cleaned datafram(df_clean).


In [ ]:
#Print the top 10 words in the bigram frequency

In [ ]:
#Write your code here to find the top 30 trigram frequency among the complaints in the cleaned datafram(df_clean).


In [ ]:
#Print the top 10 words in the trigram frequency

## The personal details of customer has been masked in the dataset with xxxx. Let's remove the masked text as this will be of no use for our analysis

In [ ]:
df_clean['Complaint_clean'] = df_clean['Complaint_clean'].str.replace('xxxx','')

In [ ]:
#All masked texts has been removed
df_clean

## Feature Extraction
Convert the raw texts to a matrix of TF-IDF features

**max_df** is used for removing terms that appear too frequently, also known as "corpus-specific stop words"
max_df = 0.95 means "ignore terms that appear in more than 95% of the complaints"

**min_df** is used for removing terms that appear too infrequently
min_df = 2 means "ignore terms that appear in less than 2 complaints"

In [ ]:
#Write your code here to initialise the TfidfVectorizer



#### Create a document term matrix using fit_transform

The contents of a document term matrix are tuples of (complaint_id,token_id) tf-idf score:
The tuples that are not there have a tf-idf score of 0

In [ ]:
#Write your code here to create the Document Term Matrix by transforming the complaints column present in df_clean.


## Topic Modelling using NMF

Non-Negative Matrix Factorization (NMF) is an unsupervised technique so there are no labeling of topics that the model will be trained on. The way it works is that, NMF decomposes (or factorizes) high-dimensional vectors into a lower-dimensional representation. These lower-dimensional vectors are non-negative which also means their coefficients are non-negative.

In this task you have to perform the following:

* Find the best number of clusters
* Apply the best number to create word clusters
* Inspect & validate the correction of each cluster wrt the complaints
* Correct the labels if needed
* Map the clusters to topics/cluster names

In [ ]:
from sklearn.decomposition import NMF

## Manual Topic Modeling
You need to do take the trial & error approach to find the best num of topics for your NMF model.

The only parameter that is required is the number of components i.e. the number of topics we want. This is the most crucial step in the whole topic modeling process and will greatly affect how good your final topics are.

In [ ]:
#Load your nmf_model with the n_components i.e 5
num_topics = #write the value you want to test out

#keep the random_state =40
nmf_model = #write your code here

In [ ]:
nmf_model.fit(dtm)
len(tfidf.get_feature_names())

In [ ]:
#Print the Top15 words for each of the topics


In [ ]:
#Create the best topic for each complaint in terms of integer value 0,1,2,3 & 4



In [ ]:
#Assign the best topic to each of the cmplaints in Topic Column

df_clean['Topic'] = #write your code to assign topics to each rows.

In [ ]:
df_clean.head()

In [ ]:
#Print the first 5 Complaint for each of the Topics
df_clean=df_clean.groupby('Topic').head(5)
df_clean.sort_values('Topic')

#### After evaluating the mapping, if the topics assigned are correct then assign these names to the relevant topic:
* Bank Account services
* Credit card or prepaid card
* Theft/Dispute Reporting
* Mortgage/Loan
* Others

In [ ]:
#Create the dictionary of Topic names and Topics

Topic_names = {   }
#Replace Topics with Topic Names
df_clean['Topic'] = df_clean['Topic'].map(Topic_names)

In [ ]:
df_clean

## Supervised model to predict any new complaints to the relevant Topics.

You have now build the model to create the topics for each complaints.Now in the below section you will use them to classify any new complaints.

Since you will be using supervised learning technique we have to convert the topic names to numbers(numpy arrays only understand numbers)

In [ ]:
#Create the dictionary again of Topic names and Topics

Topic_names = {   }
#Replace Topics with Topic Names
df_clean['Topic'] = df_clean['Topic'].map(Topic_names)

In [ ]:
df_clean

In [ ]:
#Keep the columns"complaint_what_happened" & "Topic" only in the new dataframe --> training_data
training_data=

In [ ]:
training_data

####Apply the supervised models on the training data created. In this process, you have to do the following:
* Create the vector counts using Count Vectoriser
* Transform the word vecotr to tf-idf
* Create the train & test data using the train_test_split on the tf-idf & topics


In [ ]:

#Write your code to get the Vector count


#Write your code here to transform the word vector to tf-idf

You have to try atleast 3 models on the train & test data from these options:
* Logistic regression
* Decision Tree
* Random Forest
* Naive Bayes (optional)

**Using the required evaluation metrics judge the tried models and select the ones performing the best**

In [ ]:
# Write your code here to build any 3 models and evaluate them using the required metrics



